In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# https://discuss.pytorch.org/t/how-to-implement-keras-layers-core-lambda-in-pytorch/5903
class BestMeanPooling(nn.Module):
    
  def __init__(self, n_topN):
    super(BestMeanPooling, self).__init__()
    self.topN = n_topN
        
  def forward(self, aX_tr):
    """
    
    """
    aX_tr_sorted, _ = torch.sort(aX_tr, axis=1) # also return index
    return torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("B = ", aX_tr_sorted.size(), aX_tr_sorted)
    # c = torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("C = ", c.size(), c)
    return c

# Test
aA = torch.randn(1, 200).reshape(4, 10, 5)
mean_pool = BestMeanPooling(2) 
aA = mean_pool(aA)
print(aA)

tensor([[0.9378, 1.8699, 0.5784, 0.9926, 0.9102],
        [1.1373, 1.1477, 0.5278, 1.7049, 1.1137],
        [0.7076, 1.4236, 1.3848, 1.7648, 1.3178],
        [1.8808, 2.3128, 1.0802, 1.2985, 1.7595]])


In [3]:
class SyntheticData(Dataset):
    def __init__(self, upper, x, y, z):
      self.aX_tr_sy = np.random.random_sample(upper).reshape(x, y, z)
      self.ay_tr_sy = np.random.randint(0, 2, x)

    def __len__(self):
        return len(self.aX_tr_sy)

    def __getitem__(self, idx):
        return self.aX_tr_sy[idx], self.ay_tr_sy[idx] #.squeeze(1)

In [4]:
class MassCNN(nn.Module):

  def __init__(self, n_in, n_out, num_event=10, topN=5, prob=0.00):
    super(MassCNN,self).__init__()
    self.n_filter = n_out

    #first layers
    self.conv_1        = nn.Conv1d(n_in, n_out, kernel_size=1)
    
    # output of the layer is # of filters
    self.bmpool_1      = BestMeanPooling(topN)
    self.dropout_L1    = nn.Dropout(p=prob)
    
    # Fully connected layer
    self.fcn_1 = nn.Linear(3, 2)
    # self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, aX_tr):
    aX_tr = self.conv_1(aX_tr)
    aX_tr = torch.relu(aX_tr)
    
    # This will collapse one dimension
    aX_tr = self.bmpool_1(aX_tr)
    
    if self.n_filter > 4: 
      aX_tr = self.dropout_L1(aX_tr)
    
    aX_tr = aX_tr.view(aX_tr.size(0), -1)
    aX_tr = self.fcn_1(aX_tr)
    # aX_tr = self.softmax(aX_tr)
    return aX_tr

In [5]:
def train(n_epochs=10, lr=0.1, filter=12):
    """
      Aim:

      params:

      returns:
    """
    # Random data
    d_trains = SyntheticData(750, 5, 50, 3)
    d_valids = SyntheticData(750, 5, 50, 3)
    d_tests  = SyntheticData(750, 5, 50, 3)
    
    # 
    massModel = MassCNN(50, filter, 3, 5)
    
    # Trying to remove this type casting from here.
    massModel = massModel #.float()
    
    # optimization
    optimizer = torch.optim.SGD(massModel.parameters(), lr=0.0001)
    fcost     = torch.nn.CrossEntropyLoss()
  
    cost_per_epoch = []
    
    aTrs = DataLoader(d_trains, batch_size=100, shuffle=True)
    aVas = DataLoader(d_valids, batch_size=10, shuffle=True)
    
    for epoch in range(n_epochs):
      fgr_cost = 0
      for x, y in aTrs:
        ## Back propagation the loss
        optimizer.zero_grad()
        y_pr  = massModel(x.float())
        loss  = fcost(y_pr, y)
        loss.backward()
        optimizer.step() # back propagate
        fgr_cost += loss.data
      cost_per_epoch.append(fgr_cost)
    
      correct=0
      for x_va, y_va in aVas:
        z  = massModel(x_va.float()).data
        temp, yhat = torch.max(z.data, 1)
        correct += (yhat == y_va).sum().item()
      accuracy = correct / 1000
      print(epoch, accuracy)
            
      # get accuracy list 
    #return f_ac_va



In [6]:
train()

0 0.002
1 0.002
2 0.002
3 0.002
4 0.002
5 0.002
6 0.002
7 0.002
8 0.002
9 0.002


### Keras

In [7]:
from keras.layers import Input, Dense, Lambda, Activation, Dropout
from keras.layers.convolutional import Convolution1D
from keras.models import Model
from keras.optimizers import Adam

# the input layer
ncell = 20
nmark = 5
data_input = Input(shape=(ncell, nmark))

nfilter = 10
# the filters
print(data_input.shape)
conv = Convolution1D(nfilter, 1, activation='linear', name='conv1')(data_input)
print(conv.shape)


import keras.backend as K
K.print_tensor(conv)

Using TensorFlow backend.


(None, 20, 5)
(None, 20, 10)


<tf.Tensor 'Identity:0' shape=(None, 20, 10) dtype=float32>

In [8]:
a = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=3)

In [9]:
"""
in_channel = 16
out_channel = 33
filter/kernal = 3
batch = 20
"""

m = nn.Conv1d(3, 6, 2, stride=1)
input = torch.randn(5, 3, 4)
print(input.shape)
output = m(input)
print(output.shape)

torch.Size([5, 3, 4])
torch.Size([5, 6, 3])


In [10]:
output.shape

torch.Size([5, 6, 3])

In [11]:
input

tensor([[[ 0.7028, -0.6217,  0.4973, -2.2627],
         [ 0.4807,  0.8496, -0.9030, -0.5538],
         [ 1.9475,  0.6171, -0.7366, -0.2498]],

        [[ 0.6387, -0.5125, -1.0771, -0.9551],
         [-0.5356,  0.9956,  0.0584,  2.2914],
         [-1.5711, -0.9820, -0.0799,  3.0789]],

        [[-1.0428, -1.6745, -0.7521,  0.0911],
         [ 0.3217, -0.2753,  1.3601,  0.6509],
         [ 1.0648, -2.2526, -0.4646, -0.7599]],

        [[ 0.2886,  0.4290,  0.1780, -0.8287],
         [-0.0234, -0.7935, -1.6031, -0.0356],
         [ 2.6638, -0.7362,  1.1084,  1.2157]],

        [[ 2.1391,  0.4881, -0.3508,  0.7002],
         [-1.2184,  1.3413, -1.0789, -0.1223],
         [-0.9552, -0.6782, -1.9579,  0.7080]]])

In [12]:
output

tensor([[[ 2.1031e-01,  1.0137e+00, -3.1264e-01],
         [ 5.8147e-01,  8.2541e-01, -1.3561e-01],
         [-1.8212e-01,  1.0984e-01, -5.3305e-01],
         [ 6.0273e-01,  1.2084e-01, -4.7400e-01],
         [ 9.2841e-01,  5.8898e-02, -1.8112e-01],
         [ 5.4575e-01, -1.2519e-01,  1.5662e-01]],

        [[-5.7114e-01, -2.7715e-01, -1.1713e+00],
         [-5.2369e-01, -4.3964e-01, -5.6885e-01],
         [-4.7010e-01, -2.9353e-01,  7.1560e-01],
         [ 4.7868e-02, -2.4236e-01,  1.1398e+00],
         [-5.5377e-01, -6.1071e-01,  9.3415e-01],
         [ 2.7050e-01, -1.4577e-01,  1.0217e+00]],

        [[ 1.2452e+00, -5.4066e-01,  1.8155e-01],
         [ 3.7294e-01, -9.6781e-01, -2.0210e-01],
         [-1.2265e+00, -2.0285e-01, -2.8817e-01],
         [-8.2457e-01, -4.9723e-02,  5.2548e-02],
         [-7.5778e-01, -1.0015e+00, -6.2687e-01],
         [ 2.9532e-02,  3.5118e-01, -6.4301e-02]],

        [[ 1.5545e+00, -4.4594e-02,  1.6927e-01],
         [ 1.6631e+00,  7.4171e-01,  8.5054e